In [1]:
import os
import random
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import albumentations as A

# ----------------------------
# 기본 설정
# ----------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

CSV_PATH = "/root/cv_project/datasets/data/train.csv"          # 원본 csv (ID, target)
IMG_DIR  = "/root/cv_project/datasets/data/train"              # 원본 이미지 폴더
OUT_DIR  = "/root/cv_project/datasets/data/train_offline_aug"  # 증강 이미지 폴더
OUT_CSV  = "/root/cv_project/datasets/data/train_offline_aug.csv"  # 원본+증강 병합 csv

os.makedirs(OUT_DIR, exist_ok=True)

# ----------------------------
# 클래스별 목표 개수 설정
#  - 3,7,14번은 500장, 나머지는 300장
# ----------------------------
BASE_TARGET = 300
SPECIAL_TARGETS = {3: 500, 7: 500, 14: 500}

# ----------------------------
# 증강 파이프라인 (문서 친화, 보수적)
# ----------------------------
AUG = A.Compose([
    A.Resize(384, 384, interpolation=cv2.INTER_CUBIC),
    A.Affine(scale=(0.96, 1.04), translate_percent=(0.02, 0.06),
             rotate=(-7, 7), shear=(-3, 3), p=0.8),
    A.Perspective(scale=(0.02, 0.05), p=0.3),
    A.OneOf([
        A.RandomBrightnessContrast(0.2, 0.2),
        A.CLAHE(clip_limit=(1.0, 2.0))
    ], p=0.7),
    A.OneOf([
        A.MotionBlur(blur_limit=3),
        A.MedianBlur(blur_limit=3),
        A.GaussianBlur(blur_limit=(3,3)),
        A.GaussNoise(var_limit=(5.0, 15.0)),
        A.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5)),
        A.ImageCompression(quality_lower=60, quality_upper=90),
    ], p=0.5),
], p=1.0)

# ----------------------------
# 유틸
# ----------------------------
def read_image(path):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    if img is None:
        raise FileNotFoundError(path)
    return img

def write_image(path, img):
    ok = cv2.imwrite(path, img, [cv2.IMWRITE_JPEG_QUALITY, 92])
    if not ok:
        raise IOError(path)

# ----------------------------
# 데이터 로드
# ----------------------------
df = pd.read_csv(CSV_PATH)
assert set(["ID", "target"]).issubset(df.columns), "train.csv는 ID, target 컬럼이 필요합니다."
print("원본 데이터:", df.shape)
print("원본 클래스 분포:\n", df["target"].value_counts().sort_index())

# 각 클래스 목표치 맵 구성
classes = sorted(df["target"].unique().tolist())
TARGET_MAP = {c: SPECIAL_TARGETS.get(c, BASE_TARGET) for c in classes}

# ----------------------------
# 증강 실행
# ----------------------------
aug_rows = []

for cls_id in classes:
    sub = df[df["target"] == cls_id].reset_index(drop=True)
    cur_count = len(sub)
    target_count = TARGET_MAP[cls_id]
    if cur_count >= target_count:
        print(f"[스킵] 클래스 {cls_id}: {cur_count} >= {target_count}")
        continue

    need = target_count - cur_count
    print(f"[증강] 클래스 {cls_id}: {cur_count} → {target_count} (추가 {need})")

    src_ids = sub["ID"].tolist()
    pbar = tqdm(range(need), desc=f"Aug[{cls_id}]", ncols=100)

    for i in pbar:
        src_id = src_ids[i % cur_count]  # 라운드로빈
        src_path = os.path.join(IMG_DIR, src_id)

        try:
            img = read_image(src_path)
            # (선택) 아주 가끔 90/180/270 회전 섞기 — 문서 방향 다양성 확보(확률 낮게)
            if random.random() < 0.12:
                angle = random.choice([90, 180, 270])
                if angle == 90:
                    img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
                elif angle == 180:
                    img = cv2.rotate(img, cv2.ROTATE_180)
                else:
                    img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)

            aug_img = AUG(image=img)["image"]

            stem, ext = os.path.splitext(os.path.basename(src_id))
            # 파일명: 원본명_c{cls}_aug{글로벌번호}.jpg (중복 방지)
            aug_id = f"{stem}_c{cls_id}_aug_{i:05d}{ext}"
            out_rel = os.path.join("train_offline_aug", aug_id)  # CSV용 상대경로
            out_abs = os.path.join(OUT_DIR, aug_id)             # 실제 저장 경로

            write_image(out_abs, aug_img)
            aug_rows.append({"ID": out_rel, "target": cls_id})

        except Exception as e:
            pbar.write(f"[오류] {src_id}: {e}")
            continue

print(f"증강 생성 개수: {len(aug_rows)}")

# ----------------------------
# CSV 병합 & 저장 (원본 + 증강)
# ----------------------------
df_aug = pd.DataFrame(aug_rows)
df_merged = pd.concat([df, df_aug], axis=0, ignore_index=True)
df_merged.to_csv(OUT_CSV, index=False, encoding="utf-8")

print(f"✅ 저장 완료: {OUT_CSV}")
print("최종 클래스 분포:\n", df_merged['target'].value_counts().sort_index())
print("증강 이미지 폴더:", OUT_DIR)


/root/cv_project/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_1771685/3168649556.py:46: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(5.0, 15.0)),
/tmp/ipykernel_1771685/3168649556.py:48: UserWarning: Argument(s) 'quality_lower, quality_upper' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=60, quality_upper=90),


원본 데이터: (1570, 2)
원본 클래스 분포:
 target
0     100
1      46
2     100
3     100
4     100
5     100
6     100
7     100
8     100
9     100
10    100
11    100
12    100
13     74
14     50
15    100
16    100
Name: count, dtype: int64
[증강] 클래스 0: 100 → 300 (추가 200)


Aug[0]: 100%|████████████████████████████████████████████████████| 200/200 [00:00<00:00, 208.43it/s]


[증강] 클래스 1: 46 → 300 (추가 254)


Aug[1]: 100%|████████████████████████████████████████████████████| 254/254 [00:01<00:00, 214.06it/s]


...

[증강] 클래스 15: 100 → 300 (추가 200)


Aug[15]: 100%|███████████████████████████████████████████████████| 200/200 [00:00<00:00, 203.65it/s]


[증강] 클래스 16: 100 → 300 (추가 200)


Aug[16]: 100%|███████████████████████████████████████████████████| 200/200 [00:00<00:00, 201.72it/s]

증강 생성 개수: 4130
✅ 저장 완료: /root/cv_project/datasets/data/train_offline_aug.csv
최종 클래스 분포:
 target
0     300
1     300
2     300
3     500
4     300
5     300
6     300
7     500
8     300
9     300
10    300
11    300
12    300
13    300
14    500
15    300
16    300
Name: count, dtype: int64
증강 이미지 폴더: /root/cv_project/datasets/data/train_offline_aug
